In [ ]:
!pip install tensorflow
!pip install gensim
!pip install googletrans

In [ ]:
import numpy as np

import os
import sys
import time
import re
import json
import datetime

from gensim.models import KeyedVectors
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import googletrans
from googletrans import Translator

import matplotlib.pyplot as plt

In [ ]:
print('Python version: %s' % sys.version)
print('Numpy version: %s' % np.__version__)
print('Tensorflow version: %s' % tf.__version__)
print('Googletrans version: %s' % googletrans.__version__)

# Готовые модели русского языка

## Русский язык лемматизированные слова, d=300

Скачать векторное вложение для русского языка по ссылке https://rusvectores.org/static/models/rusvectores4/unigrams/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz
```
d=300, window=5, skipgram
```

### Задание 1.a
Необходимо реализовать метод подгружающий модель из файла. Файлы в текстовом формате загружаются гораздо дольше, поэтому лучше из сразу переводить в бинарный формат.
* если расширение файла `.bin`, т.е. модель в бинарном формате, загрузить и вернуть
* если расширение файлп `.vec`, т.е. модель в текстовом формате, то
  * проверить, есть ли уже такой же файл с расширением `.bin`, если да, то загрузить и вернуть его
  * если нет, то загрузить файл в формате `.vec`
  * полученную модель сохранить в бинарном формате, имя файла не меняется, расширение меняетя на `.bin`
  * вернуть модель
  
  
```
KeyedVectors.load_word2vec_format
```

In [ ]:
def get_model(model_file):
    # your code
    pass

In [ ]:
rusvectores = get_model("""путь к скаченной модели""")

### Задание 1.б
Построить гистограмму длин векторов для модели и убедиться, что они нормированы
```
rusvectores.get_vector
plt.hist
```

### Задание 1.в
Вектора в модели упорядочены по частоте. Вывести 20 наиболее частых слов
```
rusvectores.index2word
```

## Английский язык, d=300

Скачать вектрное вложение для английского языка по ссылке http://nlp.stanford.edu/data/glove.6B.zip
```
d=300, GloVe
```

Формат скачанное вложения не совпадает с форматом gensim, необхолдимо преобразовать скачанный файл. Скачанный файл `glove.6B.300d.txt`, преобразованный файл `gensim.glove.6B.300d.txt`

In [ ]:
with open('glove.6B.300d.txt', 'r', encoding='utf-8') as f:
    count = 0
    while True:
        line = f.readline()
        if not line:
            break
        count += 1
gensim_glove_file = open('gensim.glove.6B.300d.txt', 'w', encoding='utf-8')
with open('glove.6B.300d.txt', 'r', encoding='utf-8') as f:
    gensim_glove_file.writelines('%d 300\n' % count)
    while True:
        line = f.readline()
        if not line:
            break
        gensim_glove_file.writelines(line)
gensim_glove_file.close()

In [ ]:
glove6B = get_model('gensim.glove.6B.300d.txt')

### Задание 1.г
* Убедиться, что вектора в модели GloVe не нормированы построив гистограмму длин векторов
* Построить словарь нормированных векторов `{слово: нормированный_вектор}`
```
np.linalg.norm
```
* Слова в модели упорядочены по частоте, вывести 20 наиболее частых слов

In [ ]:
glove6B_normed = # your code

### Задание 1.д
Задача аналогий. Построить решение задачи аналогий с помощью gensim, выводим 10 наиболее близких слов
* король - мужчина + женщиена = королева
* Москва - Россия + Франция = Париж
* придумать еще 10-15 аналогий

Описать результат
```
rusvectores.most_similar
glove6B.most_similar
```

## Создадим словарь с помощью Google Translate API

In [ ]:
def create_dict(filename, words, num_words=500, num_attempts=10, polite_delay=0.25, ban_deplay=10, tagged=True):
    
    if os.path.exists(filename):
        
        with open(filename, 'r', encoding='utf-8') as file:
            translations = json.load(file)
        offset = len(translations)
            
    else:
        translator = Translator()
        pattern_en = re.compile('^[a-z]+$')
        pattern_ru = re.compile('^[а-яё]+_.*') if tagged else re.compile('^[а-яё]+')
        translations = {}
        progbar = tf.keras.utils.Progbar(num_words)
        for offset, w in enumerate(words):
            if pattern_ru.match(w):
                # 
                time.sleep(polite_delay)
                success = False
                for _ in range(num_attempts):
                    try:
                        w_en = translator.translate(w.split('_')[0] if tagged else w, src='ru', dest='en').text
                        success = True
                        break
                    except:
                        time.sleep(ban_deplay)
                assert success, 'After %d attempts translation stil fails' % num_attempts
                if pattern_en.match(w_en) and w_en in glove6B.index2word:
                    translations[w] = w_en
                    progbar.add(1)
            if len(translations) >= num_words:
                break
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(translations, file, indent=4)
    
    return translations, offset

In [ ]:
rusvectores_dict, rusvectores_offset = create_dict('rusvectores_dict.json', words=rusvectores.index2word, num_words=5000)

Фиксируем значение `rusvectores_offset`, оно нам пригодится!

# Матрица перевода, точное решение

### Задание 2.a
Используя созданный словарь `rusvectores_dict` и векторные модели для русского и английского языков построить точное решение для ортогональный матрицы переводов
```
np.linalg.svd
```

In [ ]:
W = # your code

### Задание 2.б
* Построить функцию перевода, которая с помощью точного решения задачи перевода `W` и модели английского языка находит `topn` наиболее близких переводов. Функция перевода должна показывать, содержится ли слово в тренеровочном словаре `rusvectores_dict`, и корректно обрабатывать случай, когда предложенное слово не содержится в модели `rusvectores`, а значит для него невозможно подобрать векторное представление
* Привести несколько примеров перевода слов, не содержащихся в треневорочном словаре `rusvectores_dict`
* Построить словарь `translated_dict` {русское слово: [перевод1, перевод2, ..., перевод10]} отображающий каждое русское слово из словаря `rusvectores_dict` в 10 наиболее близких перводов с помощью функции перевода `translate`
```
rusvectores.get_vector
np.dot
glove6B.most_similar
```

In [ ]:
def translate(word, W, topn=10):
    # your code
    pass

In [ ]:
translated_dict = # your code

### Задание 2.в
С помощью словаря переводов, построенного с помощью Google API `rusvectores_dict` вычисляем `Precision@1`, `Precision@2`, ..., `Precision@10`, где `Presicion@n` это среднее число попаданий правильного перевода в top-n из словаря `translated_dict`. Строим график зависимости `Presicion@n` от n

Создаём валидационный словарь, который состоит из слов, не вошедших в словарь, с помощью которого была построена матрица перевода

In [ ]:
rusvectores_val_dict = create_dict('rusvectores_val_dict.json', words=rusvectores.index2word[:rusvectores_offset], num_words=500)

### Задание 2.г
* Для каждого русского слов из словаря `rusvectores_val_dict` с помощью функции `translate` строим словарь `translated_val_dict` с 10 наиболее близкими переводами, аналогично заданию 2.в
* С помощью валидационного словаря переводов `rusvectores_val_dict` вычисляем `Precision@1`, `Precision@2`, ..., `Precision@10` 
* Строим график зависимости `Presicion@n` от n для тренеровочной выборки (значения получены в задании 2.в) и для валидационной выборке

In [ ]:
translated_val_dict = # your code

### Задание 2.д
Определить функцию, принимающую 2 набора многомерных векторов одной размерности и отображающее их на двумерной плоскости используя в качестве проекции метод t-SNE. Вектора из различных множеств должны быть окрашены в разные цвета, например, красный и зелёный. Название графика должно задаваться параметром `title`, если задан параметр `path`, график должен сохраняться по заданному пути.
```
plt.title
plt.scatter
plt.savefig (не забыть после вызова plt.savefig(path) закрыть картинку вызовом plt.close()
```

In [ ]:
def plot_embeddings(vectors_ru, vectors_en, title, path=None):
    # your code
    pass

Отобразим на плоскости множество, состоящее из векторов для 500 наиболее частых слов русского языка, и наиболее частых слов английского языка

In [ ]:
plot_embeddings(embedding_ru[:500], embedding_en[:500], 'Raw embeddings')

Отобразим на плоскости множество, состоящее из векторов для 500 наиболее частых слов русского языка, **переведённых с помощью матрицы W**, и наиболее частых слов английского языка

In [ ]:
plot_embeddings(np.dot(embedding_ru[:500], W.T), embedding_en[:500], 'Translated with W')

# Матрица перевода, простая регрессия

### Задание 3.а
Используя keras API создать модель, состоящую из входящего слоя размерности равной размерности векторов для русского языка, и выходного слоя размерности равной размерности английского языка, без bias и функции активации. В качестве инициализации для kernel возьмём единичную матрицу.
```
tf.keras.model.Sequential
tf.keras.layers.Input
tf.keras.layers.Dense
tf.keras.initializers.Constant
tf.eye
```

In [ ]:
translator_reg = # your code

In [ ]:
translator_reg.compile(optimizer=tf.keras.optimizers.Adam())
translator_reg.summary()

### Задание 3.б
Используя словари `rusvectores_dict` и `rusvectores_val_dict` построить тренеровочную и тестовую выборки

In [ ]:
X, X_val = # your code
y, y_val = # your code

### Задание 3.в 
* Обучить модель на построенных выборках, контроллируя среднеквадратичную ошибку на валидационных данных, 10 эпох
* Аналогично заданию 2.б определить функцию перевода `translate_reg` использующую модель `translator_reg`
* Насколько хорошо работает этот переводчик? Привести примеры удачных и неудачных переводов
```
tf.keras.model.Model.fit
```

In [ ]:
def translate_reg(word, topn=10):
    # your code
    pass

# Состязательно обучение

Wiki word vectors https://wikipedia2vec.github.io/wikipedia2vec/pretrained/

## Русский язык, d=100
Скачать векторное вложение для русского языка по ссылке http://wikipedia2vec.s3.amazonaws.com/models/ru/2018-04-20/ruwiki_20180420_100d.txt.bz2
```
d=100, window=5, iteration=10, negative=15
```

In [ ]:
ruwiki100 = get_model('ruwiki_20180420_100d.txt')

## Английский язык, d=100

Скачать векторное вложение для английского языка по ссылке http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_100d.txt.bz2
```
d=100, window=5, iteration=10, negative=15
```

In [ ]:
enwiki100 = get_model('enwiki_20180420_100d.txt')

### Задание 4.а
В скачанных моделях слова упорядочены по частотности. Однако, некоторые слова не являются словами, а являются сущностиями из википедии. Есть так же и мусор. 
* необходимо для русской и английской википедии составить списки настоящих слов, т.е. для русской википедии слова должны состоять только из кириллических символов, а для английских - только из латиницы. 
* отдельно составить для каждой из википедий матрицу вложений для отобранных слов
* проверить, что вектора нормированы, и, при необходимости, отнормировать их

In [ ]:
words_wiki_ru = # your code
words_wiki_en = # your_code
embedding_ru = # your_code, embedding_ru.shape = [len(words_wiki_ru), 100]
embedding_en = # your_code, embedding_en.shape = [len(words_wiki_en), 100]

### Задание 4.б
Поскольку теперь мы будем строить систему перевода без учителя, мы не можем пользоваться словарём. Вместо этого определим расстояние между двумя множествами:
1. вектора переведённых русских слов
2. вектора слов английского языка

Будем ориентироваться на результат задания 2.д. Расстояние определяем следующим образом:
1. для каждой пары векторов, пришедшей из разных множеств, вычисляем косинусное расстояние, получаем матрицу размера n * m, где n - количество векторов русских слов, m - количество векторов английских слов
2. для каждой строчки этой матрицы находим максимальное значение, это косинусное расстояние между переведённым русским словом и ближайшим к нему английским словом
3. усредняем по всем полученным максимальным значениям

In [ ]:
def similarity(vectors_ru, vectors_en):
    # your code
    pass

### Задание 4.в
Используя keras API определить 
* Модель для дискриминатора, состоящую из
  * входного слоя размерности 100
  * слоя Dropout с rate=0.1
  * скрытого слоя размерности 512 и активацией relu
  * выходного размерности 1 с линейной активацией
* Модель для переводчика, точно так же, как в Задании 3.а
```
tf.keras.layers.Dropout
```

In [ ]:
discriminator = # your code

In [ ]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam())
discriminator.summary()

In [ ]:
translator_adv = # your code

In [ ]:
translator_adv.compile(optimizer=tf.keras.optimizers.Adam())
translator_adv.summary()

### Задание 4.г
Используя `embedding_ru` и `embedding_en` полученные в задании 4.а построить 2 датасета, выдающего батчи для вложения 5000 наиболее частых русских слов, и для вложений 5000 наиболее частых английских слов. Размер батча 128, и не забыть перемешать слова! 
```
tf.data.Dataset.from_tensor_slices
```

In [ ]:
dataset_ru = # your code
dataset_ru = # your code

Для обучения мы будем использовать итераторы

In [ ]:
iter_ru = iter(dataset_ru)
iter_en = iter(dataset_en)

### Задание 4.д
Определить метод одного шага совместного обучения переводчика и дискриминатора. 
* для обучения переводчика использовать функцию потерь бинарной кросс-энтропии со сглаживанием равным 0.2, `tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.2)`, для обучения переводчика - без сглаживания. 
* при обучении дискриминатора указать для дискриминатора `training=True` чтобы сработала Dropout рагуляризация, При обучении переводчика указываем для дискриминатора `training=False`.
* при обучении переводчика необходимо добавить к кросс-энтропии регуляризацию, штрафующую матрицу перевода за неортогональность вида $loss\_reg = ||1 - W\cdot W^T||^2$ с коэффициентом `ort_reg`

Во время отладки можно закомментировать аннотацию AutoGraph `@tf.function`
```
tf.keras.losses.BinaryCrossentropy
tf.function
```

In [ ]:
binary_cross_entropy_smooth = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.2)
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
@tf.function
train_step(ort_reg=1):
    v_ru = next(iter_ru)
    v_en = next(iter_en)
    # discriminator training, your code
    
    vt_ru = next(iter_ru)
    # translator trainign, your code
    
    # loss_d - discriminator training loss
    # loss_t - translator training loss
    # loss_reg - non-orthogonality loss, loss_reg = ||1 - W*W.T||^2 (L2 norm)
    # prediction_en - discriminator prediction for v_en
    # prediction_ru - discriminator prediction for translator_adv(v_ru)
    return loss_d, loss_t, loss_reg, prediction_en, prediction_ru

## Обучение

In [ ]:
n_steps = 100000
top_val = 500

### Задание 4.е
Обучить модель, из сохранённых весов отобрать значения, при котором `similarity` максимально, и посмотреть несколько вариантов перевода. 
Может потребоваться несколько запусков, иногда модель сваливается в локальный минимум и дальше не обучается.
* С помощью Tensorboard получить графики обучения для `accuracy_d_train` и `similarity`
* Аналогично заданию 2.б определить функцию перевода `translate_adv` использующую модель `translator_adv`
* Насколько хорошо удалось обучить модель? С какими словами (хотя бы в смысле top 10) она справилась?
* Как менялось взаимное расположение множеств векторов английских слов и переведённых русских (использовать сохранённые в процессе тренировки графики)? Какой можно сделать вывод?

In [ ]:
# model name
name = 'adversarial_{}'.format(datetime.datetime.now().strftime('%Y%m%d%H%M%S'))

# write tensorboard readable summary
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
writer = tf.summary.create_file_writer(os.path.join(log_dir, name))

fig_dir = os.path.join('figs', name)
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
    
model_dir = os.path.join('models', name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
log_every = 100
fig_every = 1000
save_every = 1000

history = {'step': [], 'loss_d': [], 'loss_t': [], 'accuracy_d_train': [], 'similarity': [], 'loss_reg': []}

progbar = tf.keras.utils.Progbar(n_steps, stateful_metrics=list(history.keys()))
step = 0
for _ in range(n_steps):
    
    loss_d, loss_t, loss_reg, prediction_en, prediction_ru = train_step(ort_reg=1)
    # log to tensorboard
    if step % log_every == 0:
        # write tensorboard logs
        logs = [
            ('loss_d', loss_d.numpy()), 
            ('loss_t', loss_t.numpy()), 
            ('accuracy_d_train', (np.mean(prediction_en > 0) + np.mean(prediction_ru < 0)) / 2),
            ('similarity', similarity(translator_adv(embedding_ru[:top_val]).numpy(), embedding_en[:top_val])),
            ('loss_reg', loss_reg.numpy())
        ]
        with writer.as_default():
            for name, val in logs:
                tf.summary.scalar(name, val, step=step)
                history[name] += [val]
        history['step'] += [step]
        progbar.update(step, values=logs)
    if step % fig_every == 0:
        _ = plot_embeddings(translator_adv(embedding_ru[:top_val]).numpy(), 
                            embedding_en[:top_val], 
                            'Top %d Translated at step %d' % (top_val, step), 
                            path=os.path.join(fig_dir, '%d.png' % step))
    if step % save_every == 0:
        translator_adv.save_weights(os.path.join(model_dir, 'translator-%d.h5' % step))
        discriminator.save_weights(os.path.join(model_dir, 'discriminator-%d.h5' % step))
    step += 1
    
# save history as json file
with open(os.path.join(log_dir, '{}.json'.format(name)), 'w') as jsonfile:
    json.dump({'config': config, 'history': history}, jsonfile, indent=4)

In [ ]:
def translate_adv(word, topn=10):
    # your code
    pass